In [ ]:
import pandas as pd

# Load the clean dataset with explicit encoding
df = pd.read_csv('clean_kaggle_data_2022.csv', encoding='ISO-8859-1')
df_head = pd.read_csv('clean_kaggle_data_2022.csv', encoding='ISO-8859-1', nrows=1)

# Get the first row as column names
column_names=df_head.columns.tolist()

# Drop the first two rows (header and unit information)
df = df[2:]

# Set the column names
df.columns = column_names

# Iterate through each column starting from the second column (index 1)
response_tally = []

for column in df.columns[1:]:
    responses = df[column].value_counts().reset_index()
    responses.columns = ['Response', 'Count']
    responses['Column'] = column
    response_tally.append(responses)

# Concatenate the response tally dataframes
result_df = pd.concat(response_tally, ignore_index=True)

# Save the result to an Excel file
result_df.to_excel('numberOfResponsesPerClass.xlsx', index=False)


In [66]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel('response_tally.xlsx')

# Initialize an empty dictionary to hold string tallies
column_tally = {}

# Loop through each row in the "Column" column
for item in df['Column']:
    if item in column_tally:
        column_tally[item] += 1                        
    else:
        column_tally[item] = 1

# Convert the column tally dictionary to a DataFrame
output_df = pd.DataFrame(list(column_tally.items()), columns=['Column', 'Tally'])

# Save the DataFrame to a new Excel file
output_df.to_excel('numberOfclassesPerQuestion.xlsx', index=False)


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from category_encoders import BinaryEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2 #For feature selection


In [17]:


# Load the dataset
df = pd.read_csv('clean_kaggle_data_2022.csv')

# Handling missing values by filling 'null' with 'unknown'

df = df.drop(['Q5'], axis=1) 
df = df.drop(['Duration (in seconds)','Q29'], axis=1)
df=df.drop(0)


#Set encoding labels manually for ordinal columns
Q2_encode = {'18-21':0, '22-24':1, '25-29':2, '30-34':3, '35-39':4, '40-44':5, '45-49':6, '50-54':7, '55-59':8, '60-64':9, '65-69:':10, '70+':11}
Q11_encode = {'I have never written code':0, '<1 years':1, '1-3 years':2, '3-5 years':3, '5-10 years':4, '10-20 years':5, '20+ years':6}
Q16_encode = {'I do not use machine learning methods':0, 'Under 1 year':1, '1-2 years':2, '2-3 years':3, '3-4 years':4, '4-5 years':5, '5-10 years':6, '10-20 years':7}
Q25_encode = {'0-49 employees':0, '50-249 employees':1, '250-999 employees':2, '1000-9,999 employees':3, '10,000 or more employees':4}
Q26_encode = {'0':0, '2-Jan':1, '4-Mar':2, '9-May':3, '14-Oct':4, '15-19':5, '20+':6}
Q30_encode = {'$0 ($USD)':0, '$1-$99':1, '$100-$999':2, '$1000-$9,999':3, '$0 ($USD)':4, '$10,000-$99,999':5, '$100,000 or more ($USD)':6}
Q43_encode = {'Never':0, 'Once':1, '2-5 times':2, '6-25 times':3, 'More than 25 times':4}

#Encode ordinal columns of dataset
df['Q2'] = df['Q2'].map(Q2_encode)
df['Q11'] = df['Q11'].map(Q11_encode)
df['Q16'] = df['Q16'].map(Q16_encode)
df['Q25'] = df['Q25'].map(Q25_encode)
df['Q26'] = df['Q26'].map(Q26_encode)
df['Q30'] = df['Q30'].map(Q30_encode)
df['Q43'] = df['Q43'].map(Q43_encode)

#Fill in missing data in ordinal columns with its mean
col_ordinal = ['Q2', 'Q11', 'Q16', 'Q25', 'Q26', 'Q30', 'Q43']
for col in col_ordinal:
    df[col].fillna(df[col].mean(), inplace=True)

#Encode Q9 column from Y/N to 0 and 1 and fill null with mean
df['Q9'] = df['Q9'].replace({'No': 0, 'Yes': 1})
df['Q9'].fillna(df['Q9'].mean(), inplace=True)

#Temporarily remove ordinal and target columns
removed_col = df[['Q2', 'Q11', 'Q16', 'Q25', 'Q26', 'Q30', 'Q43', 'Q9', 'Q29_Encoded', 'Q29_buckets']]
df = df.drop(columns=['Q2', 'Q11', 'Q16', 'Q25', 'Q26', 'Q30', 'Q43', 'Q9', 'Q29_Encoded', 'Q29_buckets'])

#Do one-hot encoding on rest of dataframe
df_encoded = pd.get_dummies(df)

#Reintroduce removed columns back into the dataframe
df_encoded = pd.concat([df_encoded,removed_col], axis=1)
df_encoded.head()


C:\Users\souren\AppData\Local\Temp\ipykernel_22844\2849909595.py:12: DtypeWarning: Columns (0,15,43,57,73,88,104,118,126,132,170,200,215,248,272,281,294) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('clean_kaggle_data_2022.csv')


,Q3_Man,Q3_Nonbinary,Q3_Prefer not to say,Q3_Prefer to self-describe,Q3_Woman,Q4_Algeria,Q4_Argentina,Q4_Australia,Q4_Bangladesh,Q4_Belgium,...,Q2,Q11,Q16,Q25,Q26,Q30,Q43,Q9,Q29_Encoded,Q29_buckets
1,True,False,False,False,False,False,False,False,False,False,...,8.0,5.0,2.0,0,1,3.0,2.000000,0.514341,2.0,"20,000-29,999"
2,True,False,False,False,False,False,False,False,False,False,...,3.0,5.0,5.0,2,6,4.0,0.924315,0.514341,10.0,"100,000-124,999"
3,True,False,False,False,False,False,False,True,False,False,...,11.0,6.0,6.0,3,6,2.0,0.000000,1.000000,10.0,"100,000-124,999"
4,True,False,False,False,False,False,False,False,False,False,...,5.0,5.0,6.0,3,6,2.0,2.000000,1.000000,13.0,"200,000-299,999"
5,True,False,False,False,False,False,False,False,False,False,...,5.0,5.0,6.0,3,2,6.0,4.000000,1.000000,13.0,"200,000-299,999"


In [18]:
#Create X (features) and y (target variables) dataframes for feature selection
X = df_encoded.drop(columns=['Q29_Encoded', 'Q29_buckets'],axis=1)
y = df_encoded['Q29_Encoded']

#Calculate chi squared scores of all features
chi_scores = chi2(X,y)
chi_scores

NameError: name 'chi2' is not defined